In [1]:
# Donchian Weekly Classic Trend Following System
import gta_prices
import gta_size
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p = [48, 24, 12, 6] # Look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [3]:
# Function definitions.
def charges(date, commission):
    '''Calculate trading charges.'''
    df = pd.DataFrame(index=date, columns=['date_shift', 'charges'])
    df.date_shift = df.index.values
    df.date_shift = df.date_shift.shift()
    df.charges = np.where(df.index == df.date_shift, 0, commission)
    return df.charges

def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame()
    df['upr'] = prices.high.rolling(period).max().shift(periods=1)
    df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df

def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

def trade_summary(trade_list):
    '''Generate trade summary.'''
    frame = pd.DataFrame(columns=['trade', 'entry', 'volatility','cost', 'exit', 'days', 'profit', 'pct', 'annual'])
    if trade_list.entry.iloc[-1] == 1:
        trade_num = trade_list.index.max()
    else:
        trade_num = trade_list.index.max() + 1
        
    for i in list(range(1, trade_num)):
        df = trade_list.loc[i]
        trade = df.index[0]
        entry = df.date.iloc[0]
        volatility = df.volatility.iloc[0]
        cost = df.cost.iloc[0]
        exit = df.date.iloc[-1]
        days = df.days.iloc[-1]
        profit = df.profit.sum()
        pct = (profit / cost) * 100
        annual = ((1 + pct / 100) ** (365 / days) - 1) * 100
        frame.loc[i] = [trade, entry, volatility, cost, exit, days, profit, pct, annual]
    frame = frame.set_index('trade')
    return frame

pd.set_option("display.max_columns", None)

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian channels.
for i in range(len(p)):
    globals()[f'p{i + 1}'] = p[i]
    globals()[f'dc{i + 1}'] = donchian(prices, p[i])

In [6]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = state(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = state(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = state(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [9]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = state(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [10]:
# Trade list indexed by date.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)
td.shares = gta_size.remainder_zero(td.shares, 4) # Modify number of shares to be purchased to be divisible by 4.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares), 0) # Adjust risk amount for revised share count.

In [12]:
# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [13]:
# Charges.
td['charges'] = charges(td.index, commission)

In [14]:
# Stamp duty.
td['sduty'] = np.where(td.entry==1, ((sduty / 100) * td.close * td.shares), 0)

In [15]:
# Cost (buy).
td['cost'] = np.where(td.entry == 1, ((td.close * td.shares) + td.charges + td.sduty), 0)

In [16]:
# Cost (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        cost = row['cost']
    elif row['exit'] == 1:
        td.at[index, 'cost'] = (cost / 4)

In [17]:
# Value (sell).
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges), 0)

In [18]:
# Profit.
td['profit'] = np.where(td.exit == 1, td.value - td.cost, 0)

In [19]:
# Cumulative profit.
td['profit_cum'] = td.profit.cumsum()

In [20]:
# Percentage return.
td['pct'] = np.where(td.exit == 1, ((td.profit / td.cost) * 100), 0)

In [21]:
# Trade duration.
td['days'] = 0
for index, row in td.iterrows():
    if row['entry'] == 1:
        start_date = index
    elif row['exit'] == 1:
        td.at[index, 'days'] = index - start_date
td.days = td.days.astype('timedelta64[D]')
td.days = td.days.dt.days

In [22]:
# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1) * 100)

In [23]:
# Trade list indexed by trade.
td['trade'] = td.state.cumsum()
td = td.reset_index()
td = td.set_index('trade')
td.round(2)

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,profit_cum,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2001-12-07,1.20,1.23,1.20,1.23,1.21,0.88,1.04,1,1,0,1,1,0,0.15,9914.91,8076,11.95,49.57,9976.43,0.00,0.00,0.00,0.00,0,0.00
1,2002-02-01,1.35,1.35,1.30,1.31,1.36,1.27,1.31,4,0,1,0,0,1,0.00,0.00,2019,11.95,0.00,2494.11,2629.91,135.80,135.80,5.44,56,41.28
1,2002-02-22,1.31,1.31,1.26,1.26,1.36,1.19,1.27,3,0,1,0,0,1,0.00,0.00,2019,11.95,0.00,2494.11,2525.53,31.42,167.23,1.26,77,6.11
1,2002-04-05,1.24,1.25,1.21,1.21,1.36,1.10,1.23,2,0,1,0,0,1,0.00,0.00,2019,11.95,0.00,2494.11,2440.73,-53.38,113.85,-2.14,119,-6.42
1,2002-09-06,1.18,1.18,1.14,1.15,1.36,0.99,1.17,1,0,1,0,0,1,0.00,0.00,2019,11.95,0.00,2494.11,2303.64,-190.47,-76.62,-7.64,273,-10.08
2,2003-09-05,1.13,1.17,1.12,1.17,1.14,0.79,0.96,1,1,0,1,1,0,0.18,8493.87,7256,11.95,42.47,8548.29,0.00,0.00,-76.62,0.00,0,0.00
2,2003-10-03,1.16,1.16,1.15,1.16,1.20,1.12,1.16,4,0,1,0,0,1,0.00,0.00,1814,11.95,0.00,2137.07,2088.12,-48.96,-125.57,-2.29,28,-26.07
2,2003-10-10,1.15,1.15,1.10,1.11,1.20,1.05,1.12,3,0,1,0,0,1,0.00,0.00,1814,11.95,0.00,2137.07,2000.14,-136.93,-262.51,-6.41,35,-49.87
2,2004-02-06,1.18,1.18,1.15,1.15,1.24,1.09,1.17,2,0,1,0,0,1,0.00,0.00,1814,11.95,0.00,2137.07,2066.53,-70.54,-333.05,-3.30,154,-7.65


In [24]:
# Trade summary.
tds = trade_summary(td)
tds.round({'volatility': 3, 'pct': 1, 'annual': 1})

,entry,volatility,cost,exit,days,profit,pct,annual
trade,,,,,,,,
1,2001-12-07,0.151,9976.429726,2002-09-06,273,-76.616726,-0.8,-1.0
2,2003-09-05,0.177,8548.292968,2007-07-27,1421,4119.321432,48.2,10.6
3,2009-09-11,0.242,6236.304136,2010-11-12,427,394.875464,6.3,5.4
4,2011-05-06,0.114,13234.478524,2011-08-05,91,-604.576724,-4.6,-17.1
5,2013-05-24,0.222,6790.340536,2013-10-18,147,-281.932936,-4.2,-10.0
6,2014-01-31,0.245,6181.363600,2015-10-02,609,-34.763600,-0.6,-0.3
7,2016-05-27,0.144,10507.366000,2018-03-02,644,2576.409000,24.5,13.2
8,2018-05-25,0.129,11737.646800,2018-12-14,203,620.933200,5.3,9.7
9,2019-04-05,0.181,8331.541000,2020-03-13,343,-308.096000,-3.7,-3.9


In [25]:
cost_total = tds.cost.sum()
cost_total

81543.76329

In [26]:
profit_total = tds.profit.sum()
profit_total

6405.553110000004

In [27]:
pct_total = (profit_total / cost_total) * 100
pct_total

7.855356254799611

In [28]:
days_total = (tds.exit.iloc[-1] - tds.entry.iloc[0])
days_total = days_total.days
days_total

6671

In [29]:
annual_total = ((1 + pct_total / 100) ** (365 / days_total) - 1) * 100
annual_total

0.41461235899182824